### Lab 7 - Sabal Jayaswal



In [0]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor

In [0]:
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['Price'] = pd.Series(boston.target)
training_set = df[0:404]
testing_set = df[404:506]
training_set

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,9.91655,0.0,18.10,0.0,0.693,5.852,77.8,1.5004,24.0,666.0,20.2,338.16,29.97,6.3
400,25.04610,0.0,18.10,0.0,0.693,5.987,100.0,1.5888,24.0,666.0,20.2,396.90,26.77,5.6
401,14.23620,0.0,18.10,0.0,0.693,6.343,100.0,1.5741,24.0,666.0,20.2,396.90,20.32,7.2
402,9.59571,0.0,18.10,0.0,0.693,6.404,100.0,1.6390,24.0,666.0,20.2,376.11,20.31,12.1


In [0]:
# Correlation of each column and price
def get_corr(col):
  return col.corr(training_set['Price'])

correaltions = training_set.apply(get_corr) # correlation of each value and price
correaltions = correaltions[:-1] # drop the row with price
display(correaltions)

neg = correaltions[correaltions == correaltions.min()][0]  # highest negative corr is LSTAT
pos = correaltions[correaltions == correaltions.max()][0]  # highest positive corr is RM

CRIM      -0.298169
ZN         0.313306
INDUS     -0.387645
CHAS       0.141985
NOX       -0.319973
RM         0.747334
AGE       -0.292425
DIS        0.127893
RAD       -0.229401
TAX       -0.338492
PTRATIO   -0.443601
B          0.138459
LSTAT     -0.708591
dtype: float64

In [0]:
def normalize(df, columns):
    result = df[columns]
    result = (result - result.mean())/result.std()
    return result

# NORMALIZE TESTING SET
testing_set = testing_set[['LSTAT','RM']] # only use columnn with highest pos and neg correlation
testing_set['LSTAT'] *= -1 # multiply by -1 since it has a negative correlation
testing_set = normalize(testing_set,['LSTAT','RM']) #normalize testing_set data
testing_set['Actual Price'] = df[404:506]['Price']

# NORMALIZE TRAINING SET
training_set = training_set[['LSTAT','RM']]
training_set['LSTAT'] *= -1 # multiply by -1 since it has a negative correlation
training_set = normalize(training_set,['LSTAT','RM']) #normalize testing_set data
training_set['Actual Price'] = df[0:404]['Price']

training_set

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LSTAT,RM,Actual Price
0,0.934279,0.330487,24.0
1,0.324611,0.120082,21.6
2,1.073506,1.163907,34.7
3,1.233251,0.908416,33.4
4,0.882985,1.111989,36.2
...,...,...,...
399,-2.728123,-0.657321,6.3
400,-2.259148,-0.472876,5.6
401,-1.313870,0.013514,7.2
402,-1.312405,0.096856,12.1


In [0]:
# Linear regression model prediction

pipeline = make_pipeline(PolynomialFeatures(3), LinearRegression())
pipeline.fit(training_set[['LSTAT','RM']], training_set[['Actual Price']])

testing_set['Predicted Price LR'] = pipeline.predict(testing_set[['LSTAT','RM']])

average_difference_lr = abs(testing_set['Predicted Price LR'].mean() - testing_set['Actual Price'].mean())
average_difference_lr

7.102213099265544

In [0]:
# KNeighborsRegressor prediction

# Look at the five closest neighbors.
knn = KNeighborsRegressor(n_neighbors=30)

# Fit the model on the training data.
knn.fit(training_set[['LSTAT','RM']], training_set[['Actual Price']])

# Make point predictions on the test set using the fit model.
predictions = knn.predict(testing_set[['LSTAT','RM']])

testing_set['Predicted Price KNN'] = predictions
average_difference_knn = testing_set['Predicted Price KNN'].mean() - testing_set['Actual Price'].mean()
average_difference_knn

7.29065359477125